Lesson 03: Working with Excel Files
==================

In this lesson, we will explore ways to import, process, and visualize data from an Excel file of the type that you get from the Bartington susceptometer.

Goals of this Notebook
-----------------------
1. Read and access data from multi-page Excel workbooks using Python, Numpy, and Pandas
2. Clean and pre-process common data types for more advanced use
3. Use regular expressions
4. Join fields from different data frames
5. Process magnetic susceptibility data measured using the UW Tacoma environmental geophysics lab's standard protocol
6. Understand the value of a consistent, machine-readable format for data in Excel

Packages
--------

For this tutorial, we'll be using the `numpy` package for math, the `pandas` package for data storage and manipulation, and the `matplotlib` and `seaborn` packages for making graphs. Here we load the required packages and set matplotlib up so that it makes nice figures that you can save in the `.png` format. You can change this so matplotlib saves in the `.svg` format if you'd like to edit your figures in Illustrator or Inkscape. 

In [1]:
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

%matplotlib inline
%config InlineBackend.figure_format = 'png' 

Reading Excel Files
--------------------
Excel workbooks often have multiple tabs. Our soils data has tabs for the mass measurements as well as for each set of measurements from the Bartington susceptometer. Let's read in the mass data from the sheet called *Sample info*, and store it as a pandas data frame. This only works if the first row of the sheet has titles that apply to each column. That is, if everything in the first column is the cube number, everything in the second column is the sample's ID, etc. 

In [5]:
mass=pd.read_excel("Landau Associates Soil Jan 2018.xlsx",sheet_name="Sample info")
print (mass)

   Cube #     Sample ID  Mass (g)  Mass (g) Trial 2  average mass
0       1    AGW105R-17   14.8752           14.8735      14.87435
1       2  AGW105R-28.5   14.9195           14.9170      14.91825
2       3  AGW105R-37.5   14.3339           14.3325      14.33320
3       4    AGW105R-47   11.6211           11.6182      11.61965
4       5     AGW278-23   15.2508           15.2486      15.24970
5       6     AGW278-36   16.3975           16.3918      16.39465
6       7     AGW278-55   14.9451           14.9411      14.94310
7       8     AGW278-72   16.8415           16.8406      16.84105
8       9     AGW278-84   15.1586           15.1567      15.15765
9      10    AGW278-108   18.3291           18.3259      18.32750
10     11     AGW279-12   15.4179           15.4163      15.41710
11     12     AGW279-17   17.4440           17.4385      17.44125
12     13     AGW279-20   15.5385           15.5350      15.53675
13     14     AGW280-23   15.4667           15.4644      15.46555
14    NaN 

Note that there are a couple of problems here. First of all, row 14 of the table (row 16 of the spreadsheet) is blank, resulting in a row of `NaN` ("Not a number") values. Second, notice that the standard's mass was only measured once, so there is no average mass. That's going to be a problem if we use the average mass column for sample mass in our calculations!

We need a way to fix those problems. These fixes go under the category of "data cleaning," a necessary but not very fun step of data analysis. First, we need to get rid of the blank row. We'll do that with the pandas drop *method*. A method is like an "action" that a pandas variable can do. The drop method takes two parameters: 
1. The *index* of the line you need to drop. Here, we use the index *attribute* of mass. An attribute is what it sounds like - a characteristic of that variable. Just using `mass.index` would give you the indices (row numbers) of each row. We want row 14, so we take the 14th index.
2. The `inplace=True` parameter. This tells Python to remove the 14th row from `mass` itself, not from a copy of `mass`. 

In [6]:
mass.drop(14,inplace=True)
print(mass)

Note that row 14 is now missing. Good. Let's take care of the problem with the average mass of the standard. We want to set the average mass equal to the first measurement of the standard's mass.  While we're at it, let's rename the standard to *cal_std* because that's the name we use everywhere else.

In [8]:
mass.loc[mass.index[14],'average mass'] = mass.loc[mass.index[14],'Mass (g)']
mass.loc[mass.index[14],'Sample ID'] = 'cal_std'
print(mass)

   Cube #     Sample ID  Mass (g)  Mass (g) Trial 2  average mass
0       1    AGW105R-17   14.8752           14.8735      14.87435
1       2  AGW105R-28.5   14.9195           14.9170      14.91825
2       3  AGW105R-37.5   14.3339           14.3325      14.33320
3       4    AGW105R-47   11.6211           11.6182      11.61965
4       5     AGW278-23   15.2508           15.2486      15.24970
5       6     AGW278-36   16.3975           16.3918      16.39465
6       7     AGW278-55   14.9451           14.9411      14.94310
7       8     AGW278-72   16.8415           16.8406      16.84105
8       9     AGW278-84   15.1586           15.1567      15.15765
9      10    AGW278-108   18.3291           18.3259      18.32750
10     11     AGW279-12   15.4179           15.4163      15.41710
11     12     AGW279-17   17.4440           17.4385      17.44125
12     13     AGW279-20   15.5385           15.5350      15.53675
13     14     AGW280-23   15.4667           15.4644      15.46555
15      x 

#### Now Try This
1. A few things to consider when you are saving data in Excel that you'll use with Python: First, empty lines are bad. Second, use the same names for the same samples (yes, capital letters and spaces are significant - make all of your letters either capital or lowercase, and don't use spaces!). And third, make sure that the columns in your data file are consistent. Everything in a column should be the same kind of data. *Now, as you work through this notebook, keep an Evernote note about how you are going to organize data files so you don't have these problems in the future!*   

Loading Susceptibility Data
-----------------------------

Now let's put the "Pre-Test" 10-second susceptibility data into a data frame. We're only going to print the first couple of rows of that data frame using the `head()` method. The term "head" cames from a shell command that does the same thing for text files. It's convenient for learning what kinds of data are in your data frame. 

In [25]:
susc_pre_10s=pd.read_excel("Landau Associates Soil Jan 2018.xlsx",sheet_name="Pre-tests (10s)")
print(susc_pre_10s.head())

       Sample ID   Time (h:min:s) Date (D/M/Y)  Mass Susc.Raw Meas. in SI  \
0    cal_std(LF)  20:18:09.226000   2018-12-03                   0.030237   
1    cal_std(HF)  20:19:21.286000   2018-12-03                   0.030102   
2   cal_std FD %  20:19:21.288000   2018-12-03                   0.448185   
3  AGW278-55(LF)  20:22:17.623000   2018-12-03                   0.001721   
4  AGW278-55(HF)  20:23:36.134000   2018-12-03                   0.001674   

   Mass Susc.Meas. in SI  Sample Weight in g  Correct. Factor  \
0               0.000016             18.5529         0.000539   
1               0.000016             18.5529         0.000539   
2               0.448185              0.0000         1.000000   
3               0.000001             14.9431         0.000669   
4               0.000001             14.9431         0.000669   

   Correct. Offset  Comment  
0    -2.632000e-07      NaN  
1    -9.191000e-07      NaN  
2     0.000000e+00      NaN  
3     4.871000e-07      Na

### Now Try This
1. What happens if you try printing the `tail()` instead of the head?
2. What happens if you try printing the `susc_pre_10s.columns` attribute?

Splitting up the Sample IDs
-----------------------------

If you look closely at the Sample ID column, you'll notice that it contains information about the measurement type as well as the sample's actual ID. For example, there are three rows for the standard at the beginning, corresponding to low frequency (LF), high frequency (HF), and frequency dependence (FD%) measurements. We want to separate these pieces of information. This is a bit complicated, so bear with me.

Before we start, notice that all of the information in the Sample ID column follows a certain pattern: there's the sample name, then a parenthesis or space, a two-letter code, and then some other stuff we don't care about. We want to separate the sample name and the two-letter code, and put those in two separate columns. We'll do that by first putting the info we want into a new data frame, then pasting that data frame onto our old friend `susc_pre_10s` line by line.

So, here we go. First, we'll use pandas' `str.extract()` method to extract two columns of information from the Sample ID column. The `str.extract()` method takes two parameters:
1. A *regular expression*: this is a Python-y way of specifying the text pattern we described above: 
    - The "`\S`" means "any character that isn't a blank". (If you used "`\s`", it would mean "any space, tab, or other blank character".) 
    - The "`+`" means "one or more of the character I just described". 
    - The "`[( ]`" means "either ( or a space". 
    - The "`.`" means "any ol' character".
    - The "`()`" mean "put these things in a column".
2. The `expand=True` parameter, which I'm still unsure about...

Second step: we rename the columns to something useful.

In [26]:
IDs=susc_pre_10s['Sample ID'].str.extract('(\S+)[( ](..).+',expand=True)
IDs.columns=['ID','Measurement Type']
print(IDs.head())

          ID Measurement Type
0    cal_std               LF
1    cal_std               HF
2    cal_std               FD
3  AGW278-55               LF
4  AGW278-55               HF


OK - looks good. Now we have to paste this onto our favorite data frame. We'll use pandas' `concat()` method, which is short for "concatenate", meaning "squish these two data frames together so the rows (or columns) line up". The first parameter is an array of data frames to be squished. The second parameter tells pandas how to do the squishing. In this case, we use `axis=1` to say "the rows need to line up". If we said `axis=0`, the columns would need to line up.  

In [27]:
susc_pre_10s=pd.concat([IDs,susc_pre_10s],axis=1)
print(susc_pre_10s.head())

          ID Measurement Type      Sample ID   Time (h:min:s) Date (D/M/Y)  \
0    cal_std               LF    cal_std(LF)  20:18:09.226000   2018-12-03   
1    cal_std               HF    cal_std(HF)  20:19:21.286000   2018-12-03   
2    cal_std               FD   cal_std FD %  20:19:21.288000   2018-12-03   
3  AGW278-55               LF  AGW278-55(LF)  20:22:17.623000   2018-12-03   
4  AGW278-55               HF  AGW278-55(HF)  20:23:36.134000   2018-12-03   

   Mass Susc.Raw Meas. in SI  Mass Susc.Meas. in SI  Sample Weight in g  \
0                   0.030237               0.000016             18.5529   
1                   0.030102               0.000016             18.5529   
2                   0.448185               0.448185              0.0000   
3                   0.001721               0.000001             14.9431   
4                   0.001674               0.000001             14.9431   

   Correct. Factor  Correct. Offset  Comment  
0         0.000539    -2.632000e-

### Now Try This
1. Change `axis=1` to `axis=0` and see what happens. Then change it back!

Merging Data
-------------
Now we have two data frames which tell us different information about each sample - the mass and the susceptibility measurements. We need to attach the masses somehow to the susceptibility data. We do this using a *database merge*. This essentially says, "take the mass for each sample, and attach it to the susceptibility data frame, for all the rows where the sample IDs are the same". The pandas method for this is called, oddly enough, `merge()`. It takes the following parameters:
1. The "left data frame" (in this case `susc_pre_10s`), which is usually the one with more measurements. 
2. The "right data frame" (in this case `mass`).
3. The column in the left data frame with the information that matches something in the right data frame.
4. The column in the right data frame with the information that matches something in the left data frame. Right on! Note that if there's a column with the same label in both data frames and you don't specify `left_on` and `right_on`, pandas will use whatever columns have (*exactly*) matching names.

In [29]:
susc_pre_10s=pd.merge(susc_pre_10s,mass,left_on='ID',right_on='Sample ID')
print(susc_pre_10s.head())

        ID Measurement Type   Sample ID_x   Time (h:min:s) Date (D/M/Y)  \
0  cal_std               LF   cal_std(LF)  20:18:09.226000   2018-12-03   
1  cal_std               HF   cal_std(HF)  20:19:21.286000   2018-12-03   
2  cal_std               FD  cal_std FD %  20:19:21.288000   2018-12-03   
3  cal_std               LF   cal_std(LF)  20:38:14.994000   2018-12-03   
4  cal_std               HF   cal_std(HF)  20:39:10.834000   2018-12-03   

   Mass Susc.Raw Meas. in SI  Mass Susc.Meas. in SI  Sample Weight in g  \
0                   0.030237               0.000016             18.5529   
1                   0.030102               0.000016             18.5529   
2                   0.448185               0.448185              0.0000   
3                   0.030237               0.000016             18.5529   
4                   0.030218               0.000016             18.5529   

   Correct. Factor  Correct. Offset  Comment Cube # Sample ID_y  Mass (g)  \
0         0.000539   

In [ ]:
susc_pre_10s.loc[susc_pre_10s['ID']!='cal_std','volume']=7.E-6
susc_pre_10s.loc[susc_pre_10s['ID']=='cal_std','volume']=10.E-6
susc_pre_10s['density']=(susc_pre_10s['Mass (g)']/1000.)/(susc_pre_10s['volume'])
susc_pre_10s.head()

In [ ]:
susc_pre_10s.drop(susc_pre_10s[susc_pre_10s['Measurement Type']=='FD'].index,inplace=True)
susc_pre_10s['K (SI)']=susc_pre_10s['Mass Susc.Raw Meas. in SI']*(10.E-6)/susc_pre_10s['volume']
susc_pre_10s['Chi (m3/kg)']=susc_pre_10s['K (SI)']/susc_pre_10s['density']
susc_pre_10s

In [ ]:
susc_pre_10s.groupby(['ID','Measurement Type'])['K (SI)'].agg({'mean':np.mean, 'stdev':np.std})

In [ ]:
susc_10s=pd.read_excel("Landau Associates Soil Jan 2018.xlsx",sheet_name="Tests for All Samples (10s)")
IDs=susc_10s['Sample ID'].str.extract('(\S+)[( ](..).+',expand=True)
IDs.columns=['ID','Measurement Type']
susc_10s=pd.concat([IDs,susc_10s],axis=1)
susc_10s=pd.merge(susc_10s,mass,left_on='ID',right_on='Sample ID')
susc_10s.loc[susc_10s['ID']!='cal_std','volume']=7.E-6
susc_10s.loc[susc_10s['ID']=='cal_std','volume']=10.E-6
susc_10s['density']=(susc_10s['Mass (g)']/1000.)/(susc_10s['volume'])
susc_10s.drop(susc_10s[susc_10s['Measurement Type']=='FD'].index,inplace=True)
susc_10s['K (SI)']=susc_10s['Mass Susc.Raw Meas. in SI']*(10.E-6)/susc_10s['volume']
susc_10s['Chi (m3/kg)']=susc_10s['K (SI)']/susc_10s['density']
susc_10s_avg=susc_10s.groupby(['ID','Measurement Type'])['K (SI)'].mean()

In [ ]:
susc_10s_avg=susc_10s_avg.unstack()
susc_10s_avg['FD']=susc_10s_avg.LF-susc_10s_avg.HF
susc_10s_avg['FDpercent']=100.*(susc_10s_avg.LF-susc_10s_avg.HF)/susc_10s_avg.LF
susc_10s_avg=susc_10s_avg.reset_index(level=0)

In [ ]:
fig=plt.figure(figsize=(10, 5), dpi= 80, facecolor='w', edgecolor='k')
sns.set_style("whitegrid")
grid=plt.GridSpec(2,3, hspace=0.3)
plt.subplot(grid[0,0:1])
sns.barplot(x="K (SI)", y="ID", hue="Measurement Type", data=susc_10s.sort_values('ID'))
plt.subplot(grid[0,2])
sns.barplot(x="FDpercent", y="ID", color='k', data=susc_10s_avg.sort_values('ID'))
plt.subplot(grid[1,:])
sns.jointplot(x='LF',y='FDpercent', data=susc_10s_avg[susc_10s_avg.l'ID'].sort_values('ID'));